# 1. 인공신경망의 한계와 딥러닝 출현
: 퍼셉트론은 다수의 신호를 입력으로 받아 흐른다/안흐른다(0,1)는 정보를 앞으로 전달하는 원리로 작동

- AND게이트
- OR게이트
- XOR게이트

# 2. 딥러닝 구조
<br>
<h3>1. 딥러닝 용어</h3>

- 층
    - 입력층 : 데이터를 받아들이는 층
    - 은닉층 : 모든 입력 노드부터 입력값을 받아 가중합을 계산하고, 이 값을 활성화 함수에 적용하여 출력층에 전달
    - 출력 층 신경망의 최종 결괏값이 포함
- 가중치 : 노드와 노드 간 연결 강도
- 바이어스 : 가중합에 더해주는 상수. 하나의 뉴런에서 활성화함수를 거쳐 최종적으로 출력되는 값을 조절
- 가중합, 전달 함수 : 가중치와 신호의 곱을 합한 것
- 함수
    - 활성화 함수 : 신호를 입력받아 적절히 처리하는 함수
    - 손실 함수 : 가중치 학습을 위해 출력함수의 결과와 실제 값 간의 오차를 측정하는 함수

- 활성화 함수
    - 시그모이드 함수 : 선형함수의 결과를 0~1 사이에서 비선형 형태로 변형해줌. 기울기 소멸 문제 발생하여 딥러닝 모델에서는 잘 사용 X
      $f(x) = \frac{1}{1+e^(-x)}$
    - 하이퍼볼릭탄젠트
    - 렐루 : 음수일때는 0을 출력하고 양수일 때는 x 출력. 경사하강법에 영향을 주지 않아 학습 속도가 빠르고 기울기 소멸 문제 발생 x
    - 리키 렐수 : 음수면 0이 아닌 0.001처럼 매우 작은 수를 반환
    - 소프트맥스 : 함수는 입력값을 0~1 사이에 출력되도록 정규화하여 출력값들의 총합이 항상 1이 되도록 함
     <br>  $y_k = \frac{exp(a_k)}{\sum_{i=1}^{n} exp(a_i)}$
    

```python
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net,self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden) #은닉층
        self.relu = torch.nn.ReLu(inplace=True)
        self.out = torch.nn.Linear(n_hidden, n_output) #출력층
        self.softmax = torch.nn.Softmax(dim=n_output)
    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x) #은닉층을 위한 렐루 활성화 함수
        x = self.out(x)
        x = self.softmax(x) #출력층을 위한 소프트맥스 활성화함수
        return x
```

- 손실함수
    - 평균제곱오차 $MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \widehat{y}_i)^2$
    ```python
    import torch
    
    loss_fn = torch.nn.MSELoss(reduction='sum')
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    ```
    - 크로스 엔트로피 오차 $CrossEntropy = -\sum_(i=1)^n y_i \log \widehat{y}_i$
<br>      : 분류 문제에서 원-핫 인코딩 했을 때만 사용할 수 있는 오차 계산법. 경사 하강법 과정에서 학습이 지역 최소점에서 멈추게 할 수 있음. 
    ```python
    loss = nn.CrossentropyLoss()
    input = torch.randn(5,6,requires_grad=True) #평균이 0이고 표준편차가 1인 가우시안 정규분포를 이용하여 숫자를 생성
    target = torch.empty(3, dtype=torch.long).random_(5)
    output = loss(input, target)
    output.backward()
    ```

<h3>2. 딥러닝 학습</h3>

딥러닝은 크게 순전파와 역전파 두 단계로 진행
- 순전파 : 네트워크에 훈련 데이터가 들어올 때 발생.
    - 데이터를 기반으로 예측값을 계산하기 위해 전체 신경망 교차해 지나감
- 역전파 : 손실함수로 네트워크의 예측 값과 실제 값의 차이(손실, 오차) 추정
    - 예측값과 실제 값 차이를 각 뉴런의 가중치로 미분한 후 기존 가중치 값에서 뺌

<h3>3. 딥러닝의 문제점과 해결 방안</h3>

- 과적합 문제 발생 > 드롭 아웃
- 기울기 소멸 문제 > 시그모이나 tanh 대신 렐루 함수 사용
- 성능 감소 문제 > SGD, 미니 배치 경사 하강법

- 배치 경사 하강법 $W=W-a \nabla J(W,b)$ ($a$:학습률, $J$:손실함수)
  <br>: 전체 데이터셋에 대한 오류를 구한 후 기울기를 한 번만 계산하여 모델 파라미터 업데이트. 즉, 전체 훈련 데이터셋에 대해 가중치를 편미분
- 확률적 경사 하강법(SGD) : 임의로 선택한 데이터에 대해 기울기를 계산하는 방법. 적은 데이터를 사용하기 때문에 빠른 계산 가능
    - 파라미터 변경 폭이 불안정하고, 정확도가 낮을 수 있지만 속도가 빠름
- 미니배치 경사 하강법 : 전체 데이터셋을 미니배치 여러개로 나누고 미니 배치 한 개마다 기울기를 구한 후 그것의 평균 기울기를 이용하여 모델을 업데이트 해서 학습
```python
class CustorDataset(Dataset):
    def __init__(self):
        self.x_data = [[1,2,3], [4,5,6],[7,8,9]]
        self.y_data = [[12],[18],[11]]
        def __len__(self):
            return len(self.x_data)
        def __.getitem__(self, idx):
            x = torch.FloatTensor(self.x_data[idx])
            y = torch.FloatTensor(self.y_data[idx])
            return x,y
dataset = CustomDataset()
dataloader = DataLoader(
    dataset,
    batch_size = 2, #미니배치 크기로 2의 제곱수를 사용하겠다는 의미
    shuffle = True #데이터를 불러올 때마다 랜덤으로 섞어서 가져옴
)
```

- 옵티마이저 유형
    - 속도를 조정하는 법
        - **아다그라드** :: optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01) #학습률 기본값은 1e-2
        <br>  : 가중치의 업데이트 횟수에 따라 학습률을 조정.
        <br>  : 많이 변화하는 변수들의 학습률은 작게, 적게 변화하는 변수들의 학습률은 크게
        <br>  : 파라미터마다 다른 학습률을 주기 위해 기울기 크기의 누적 G 사용
        <br>  : 기울기가 0에 수렴하는 문제가 있어 RMSFrop을 사용
        - **아다델타** :: optimizer = torch.optim.Adadelta(model.parameters(),lr=1.0)
        <br>  : 아다그라드에서 G값이 커짐에 따라 학습이 멈추는 문제를 해결
        <br>  : 학습률 -> 가중치의 변화량의 누적값(D함수) 으로 변환. 따라서 학습률 파라미터 필요X
        - **RMSProp** :: optimizer  = troch.optim.RMSProp(model.parameters(), lr=0.01)
        <br>  : 아다그라드에서 학습이 안되는 문제를 해결하기 위해 G함수에서 감마 추가 (G값이 너무 커짐을 방지)
        <br>  : 감마를 통해 사용자가 학습률 크기를 비율로 조정 가능
    - 운동량을 조정하는 방법
        - **모멘텀** :: optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        <br>  : 가중치를 수정하기 전에 수정방향 (+,-)을 참고하여 같은 방향으로 일정한 비율만 수정하는 방법
        <br>  : 지그재그 현상이 줄어들고 이전 이동 값을 고려하여 일정 비율만큼 다음 값을 결정하므로 관성 효과를 얻을 수 있다는 장점이 있음
        <br>  : 모멘텀 값을 0.9에서 시작하여 0.95, 0.99처럼 조금씩 증가시키면서 사용
        -**네스테로프 모멘텀** :: optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterove = True)
        <br>  : 모멘텀 값과 기울기 값이 더해져 실제 값을 만드는 기존 모멘텀과 달리 모멘텀 값이 적용된 지점에서만 기울기 값을 계산
        <br>  : 멈추어야 할 시점에서도 관성에 의해  더 갈 수 있는 단점이 있지만 네스테로프는 절반 정도 이동 후 어떤 방식으로 이동할지 계산하여 결정하기 때문에 단점 극복 가능
        <br>  : 즉 빠른 이속 + 적절한 제동
    - 속도와 운동량 혼용 방법
        - **아담** :: optimizer=torch.optim.Adam(model.parameters(), lr=0.01) #학습률 기본값은 1e-3
        <br>  : 모멘텀 + RMSProp

<h3>4. 딥러닝의 이점</h3>

- 특성추출 : 데이터별 특징을 토대로 데이터를 벡터로 변환하는 작업
    - 딥러닝이 활성화 되기 이전 SVM, 나이브 베이즈, 로지스틱 회귀의 특성 추출은 매우 복잡하며 전문지식 필요
    - 딥러닝 에서는 이러한 특성 추출 과정을 알고리즘에 통합
- 빅데이터의 효율적 활용 

# 3. 딥러닝 알고리즘

<h3>1. 심층 신경망</h3> : 입출력층 사이에 다수의 은닉층을 포함하는 인공 신경망

- 머신러닝에서는 비선형 분류를 위해 여러 트릭 사용
- 심층 신경망은 다수의 은닉층을 추가했기 때문에 별도의 트릭 없이 비선형 분류가 가능
- 학습을 위한 연산량이 많고 기울기 소멸 문제 등이 발생
- 이를 해결하기 위해 드롭아운, 렐루, 배치 정규화 등을 적용

<h3>2. 합성곱 신경망</h3> : 합성곱층과 풀링층을 포함하는 이미지 처리 성능이 좋은 인공 신경망 알고리즘

- 각 층의 입출력 형상을 유지
- 이미지 공간 정보를 유지하면서 인접 이미지와 차이가 있는 특징을 효과적으로 인식
- 복수 필터로 이미지의 특징을 추출하고 학습
- 추출한 이미지의 특징을 모으고 강화하는 풀링층 존재
- 필터를 공유 파라미터로 사용하기 때문에 일반 인공 신경망과 비교하여 학습 파라미터가 매우 적음

<h3>3. 순환 신경망</h3> : 시간 흐름에 따라 변화하는 데이터를 학습하기 위한 인공 신경망. 현재 결과가 이전 결과와 연관이 있음을 의미
    
- 시간성을 가진 데이터가 많다
- 시간성 정보를 이용하여 데이터의 특징을 잘 다룸
- 매우 긴 데이터를 처리하는 연구가 활발히 진행
- 그러나 기울기 소멸 문제로 학습이 제대로 되지 않는 문제 => LSTM 순환 신경망

<h3>4. 제한된 볼츠만 머신</h3> : 가시층과 은닉층으로 구성된 모델. 가시층, 은닉층간의 연결은 없고 가시층~은닉층 간의 연결만 있다

- 차원감소, 분류, 선형 회귀 분석. 협업 필터링, 특성 값 학습, 주제 모델링에 사용
- 기울기 소멸 문제를 해결하기 위해 사전학습 용도로 활용
- 심층 신뢰 신경망의 요소로 활용 (상대적으로 많이 사용 X)

<h3>5. 심층 신뢰 신경망</h3> : 입력층과 은닉층으로 구성된 제한된 볼츠만 머신을 블록처럼 여러 층으로 쌓은 형태로 연결된 신경망
    
1. 가시층과 은닉층 1에 제한된 볼츠만 머신을 사전 훈련
2. 첫 번째 층 입력 데이터와 파라미터를 고정하여 두 번째 층 제한된 볼츠만 머신을 사전훈련
3. 원하는 층 개수만큼 제한된 볼츠만 머신을 쌓아 올려 전체 DBN을 완성

- 순차적으로 심층 신뢰 신경망을 학습시켜가면서 계층적 구조 생성
- 비지도 학습으로 학습
- 위로 올라갈수록 추상적 특징 추출
- 학습된 가중치를 다층 퍼셉트론의 가중치 초기값으로 사용